In [8]:
with open('data/bsky_credentials.txt', 'r') as file:
    username = file.readline().strip()
    password = file.readline().strip()

In [ ]:
from atproto import Client, AsyncClient

client = Client()
client.login(username, password)
client.send_post(text='Hello World!')

C:\Users\celin_e3kjw6h\AppData\Local\Temp\ipykernel_11772\3146495994.py:4: RuntimeWarning: coroutine 'AsyncClient.login' was never awaited
  client.login(username, password)


<coroutine object AsyncClient.send_post at 0x000001BAC54BC0C0>

In [21]:
import requests

response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.graph.getFollows?actor=mjfree.bsky.social")
data = response.json()
data

{'follows': [{'did': 'did:plc:nb4gqdu45cmimfqjrlezw5l7',
   'handle': 'beccakittybear.bsky.social',
   'displayName': '@Beccakittybear.bsky.social',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:nb4gqdu45cmimfqjrlezw5l7/bafkreifhviglc4qlu4x6l7ulw7vh66syuxqkwxqel2lb4e4e65ejkd73za@jpeg',
   'associated': {'chat': {'allowIncoming': 'none'}},
   'labels': [],
   'createdAt': '2023-09-18T02:28:22.086Z',
   'description': 'Democrat 💙- I love America & my  Broncos  🧡🐎💙-  LGBTQ 🌈 ,Air Force Brat 🇺🇸',
   'indexedAt': '2024-12-13T23:19:02.543Z'},
  {'did': 'did:plc:gqbkejxbgymc2vpfra3idhdu',
   'handle': 'gizmosspace.bsky.social',
   'displayName': 'Jason',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gqbkejxbgymc2vpfra3idhdu/bafkreiatdauodoxy47swmhgdk5ut43rkgxuktmnaiu3spr3nom7pies3im@jpeg',
   'labels': [],
   'createdAt': '2024-08-29T20:53:42.925Z',
   'description': ' #Resister. #SlavaUkraini. I am an ex-Republican but a proud Democrat. I am happy to live in the P